In [276]:
from sklearn.datasets import fetch_openml
import pandas as pd

# Loading titanic data
titanic = fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame

print(df.shape)
df.head()

(1309, 14)


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [277]:
df = df[ ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'survived'] ]
aaa = df['age'].isna()

print(df.isnull().sum())
print(df['age'].mean())
print(df.loc[aaa].index)

pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
survived      0
dtype: int64
29.8811345124283
Index([  15,   37,   40,   46,   59,   69,   70,   74,   80,  106,
       ...
       1282, 1283, 1284, 1291, 1292, 1293, 1297, 1302, 1303, 1305],
      dtype='int64', length=263)


In [278]:
# Getting mean values of each pclass
mean_ages_by_pclass = df.groupby('pclass')['age'].transform('mean').round()

# Replacing missing ages with the class-specific mean age.
df.loc[:, 'age'] = df['age'].fillna(mean_ages_by_pclass)

# Checing if there are still any missing values of age
no_of_missing_age = df.loc[:,'age'].isna().sum()
if no_of_missing_age == 0:
    print("No missing age exists.")
elif no_of_missing_age == 1:
    print(f"There is {no_of_missing_age} missing age.")
else:
    print(f"There are {no_of_missing_age} missing ages.")

No missing age exists.


In [273]:
# Leaning model to predict survivers by age using Decision Tree. 
# I expect a single input feature would make this not accuarate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

feature_set = df[['age']]
label_set = df['survived']

# print(feature_set.head())
# print(label_set.head())

X_train, X_test, y_train, y_test = train_test_split(feature_set, label_set, test_size=0.2, random_state=11)

df_dtc = DecisionTreeClassifier(random_state=11)
df_dtc.fit(X_train, y_train)
pred = df_dtc.predict(X_test)

accuracy = accuracy_score(y_test, pred)
accuracy

0.5916030534351145

In [272]:
# Leaning model to predict survivers by age, pclass, sex using Decision Tree. 
# I expect a single input feature would make this not accuarate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df.loc[:, 'numed_sex'] = df.loc[:,'sex'].map(lambda x: 0 if x=='male' else 1).astype('int')
    
feature_set = df[['age', 'pclass', 'numed_sex']]
label_set = df['survived']

X_train, X_test, y_train, y_test = train_test_split(feature_set, label_set, test_size=0.2, random_state=11)

df_dtc = DecisionTreeClassifier(random_state=11)
df_dtc.fit(X_train, y_train)
pred = df_dtc.predict(X_test)

accuracy = accuracy_score(y_test, pred)
accuracy

0.7786259541984732

In [287]:
# with KFold (n_split=7)
from sklearn.model_selection import KFold
import numpy as np

df.loc[:, 'numed_sex'] = df.loc[:,'sex'].map(lambda x: 0 if x=='male' else 1).astype('int')
    
feature_set = df[['age', 'pclass', 'numed_sex']]
label_set = df['survived']

kfold = KFold(n_splits=7)

acc_cv = []
for tr_index, te_index in kfold.split(feature_set):
    X_train, X_test = feature_set.loc[tr_index], feature_set.loc[te_index]
    y_train, y_test = label_set.loc[tr_index], label_set.loc[te_index]

    df_dtc = DecisionTreeClassifier(random_state=2)
    df_dtc.fit(X_train, y_train)
    pred = df_dtc.predict(X_test)

    acc_cv.append(np.round(accuracy_score(y_test, pred), 3))

print("Accuracy with KFold(n_split=5):", np.mean(acc_cv))

Accuracy with KFold(n_split=5): 0.774
